# Challenge 2: Sentiment Analysis

In this challenge we will learn sentiment analysis and practice performing sentiment analysis on Twitter tweets. 

## Introduction

Sentiment analysis is to *systematically identify, extract, quantify, and study affective states and subjective information* based on texts ([reference](https://en.wikipedia.org/wiki/Sentiment_analysis)). In simple words, it's to understand whether a person is happy or unhappy in producing the piece of text. Why we (or rather, companies) care about sentiment in texts? It's because by understanding the sentiments in texts, we will be able to know if our customers are happy or unhappy about our products and services. If they are unhappy, the subsequent action is to figure out what have caused the unhappiness and make improvements.

Basic sentiment analysis only understands the *positive* or *negative* (sometimes *neutral* too) polarities of the sentiment. More advanced sentiment analysis will also consider dimensions such as agreement, subjectivity, confidence, irony, and so on. In this challenge we will conduct the basic positive vs negative sentiment analysis based on real Twitter tweets.

NLTK comes with a [sentiment analysis package](https://www.nltk.org/api/nltk.sentiment.html). This package is great for dummies to perform sentiment analysis because it requires only the textual data to make predictions. For example:

```python
>>> from nltk.sentiment.vader import SentimentIntensityAnalyzer
>>> txt = "Ironhack is a Global Tech School ranked num 2 worldwide.   Our mission is to help people transform their careers and join a thriving community of tech professionals that love what they do."
>>> analyzer = SentimentIntensityAnalyzer()
>>> analyzer.polarity_scores(txt)
{'neg': 0.0, 'neu': 0.741, 'pos': 0.259, 'compound': 0.8442}
```

In this challenge, however, you will not use NLTK's sentiment analysis package because in your Machine Learning training in the past 2 weeks you have learned how to make predictions more accurate than that. The [tweets data](https://www.kaggle.com/kazanova/sentiment140) we will be using today are already coded for the positive/negative sentiment. You will be able to use the Naïve Bayes classifier you learned in the lesson to predict the sentiment of tweets based on the labels.

## Conducting Sentiment Analysis

### Loading and Exploring Data

The dataset we'll be using today is located in the lab directory named `Sentiment140.csv.zip`. You need to unzip it into a `.csv` file. Then in the cell below, load and explore the data.

*Notes:* 

* The dataset was downloaded from [Kaggle](https://www.kaggle.com/kazanova/sentiment140). We made a slight change on the original data so that each column has a label.

* The dataset is huuuuge (1.6m tweets). When you develop your data analysis codes, you can sample a subset of the data (e.g. 20k records) so that you will save a lot of time when you test your codes.

In [1]:
# your code here
import pandas as pd
import nltk
import numpy as np


In [2]:
tweets = pd.read_csv(r'C:\Users\tiina\Downloads\Sentiment140.csv')

In [61]:
tweets.shape

(1600000, 7)

In [3]:
tweets.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [4]:
tweets = tweets.sample(20000)

### Prepare Textual Data for Sentiment Analysis

Now, apply the functions you have written in Challenge 1 to your whole data set. These functions include:

* `clean_up()`

* `tokenize()`

* `stem_and_lemmatize()`

* `remove_stopwords()`

Create a new column called `text_processed` in the dataframe to contain the processed data. At the end, your `text_processed` column should contain lists of word tokens that are cleaned up. Your data should look like below:

![Processed Data](data-cleaning-results.png)

In [5]:
# your code here
from challenge1 import clean_up, tokenize, stem_and_lemmatize, remove_stopwords


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\tiina\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\tiina\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\tiina\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [7]:
# Text proccessed 

tweets['text_processed'] = tweets.text.apply(lambda x: remove_stopwords(stem_and_lemmatize(tokenize(clean_up(x)))))

In [8]:
tweets.head()

,target,ids,date,flag,user,text,text_processed
702726,0,2255486479,Sat Jun 20 11:20:09 PDT 2009,NO_QUERY,SuzyanneCMWM,@Quiksand already!? U guys eating them now,"[quiksand, alreadi, u, guy, eat]"
1596955,4,2192810640,Tue Jun 16 07:34:58 PDT 2009,NO_QUERY,nicoleeee,"Feed the puppies, a little stretching, a.m. ru...","[feed, puppi, littl, stretch, run, good, morn,..."
157534,0,1956398530,Thu May 28 21:47:40 PDT 2009,NO_QUERY,SAMANTH0NG,having the biggest thought right now,"[biggest, thought, right]"
1425035,4,2058979183,Sat Jun 06 16:00:32 PDT 2009,NO_QUERY,eBaby74,gotta find the hole first. came home from the ...,"[got, ta, find, hole, first, came, home, store..."
583646,0,2215011422,Wed Jun 17 17:32:51 PDT 2009,NO_QUERY,JizzlePGH,so why is the weather like this... so glad i g...,"[whi, weather, like, thi, glad, got, ride, hom..."


### Creating Bag of Words

The purpose of this step is to create a [bag of words](https://en.wikipedia.org/wiki/Bag-of-words_model) from the processed data. The bag of words contains all the unique words in your whole text body (a.k.a. *corpus*) with the number of occurrence of each word. It will allow you to understand which words are the most important features across the whole corpus.

Also, you can imagine you will have a massive set of words. The less important words (i.e. those of very low number of occurrence) do not contribute much to the sentiment. Therefore, you only need to use the most important words to build your feature set in the next step. In our case, we will use the top 5,000 words with the highest frequency to build the features.

In the cell below, combine all the words in `text_processed` and calculate the frequency distribution of all words. A convenient library to calculate the term frequency distribution is NLTK's `FreqDist` class ([documentation](https://www.nltk.org/api/nltk.html#module-nltk.probability)). Then select the top 5,000 words from the frequency distribution.

In [9]:
# your code here
from nltk.probability import FreqDist
from operator import itemgetter

In [10]:
fdist = FreqDist()

for word_list in tweets.text_processed:
    for word in word_list:
        fdist[word] += 1
        


In [11]:
res = dict(sorted(fdist.items(), key = itemgetter(1), reverse = True)[:5000]) 

{'go': 138651,
 'get': 110823,
 'day': 109029,
 'wa': 105014,
 'thi': 94754,
 'good': 92537,
 'work': 87862,
 'like': 83828,
 'love': 82636,
 'quot': 73384,
 'got': 71096,
 'u': 70407,
 'today': 68709,
 'time': 66378,
 'lol': 59475,
 'thank': 59433,
 'back': 57371,
 'one': 57359,
 'want': 57335,
 'miss': 57263,
 'know': 54963,
 'see': 51355,
 'feel': 51268,
 'think': 51194,
 'im': 50685,
 'realli': 50077,
 'amp': 48767,
 'night': 45784,
 'hope': 44926,
 'watch': 43640,
 'still': 43618,
 'need': 43425,
 'make': 43215,
 'well': 42953,
 'new': 42450,
 'na': 42116,
 'home': 40825,
 'oh': 39907,
 'look': 39753,
 'ha': 39499,
 'come': 39110,
 'much': 37046,
 'last': 36266,
 'twitter': 36175,
 'morn': 35726,
 'tomorrow': 34331,
 'wish': 34024,
 'great': 33663,
 'wait': 32598,
 'sleep': 32455,
 'haha': 31537,
 'sad': 30044,
 'fun': 28713,
 'whi': 28521,
 'week': 28339,
 'tri': 28255,
 'onli': 28142,
 'right': 28113,
 'follow': 28036,
 'happi': 27715,
 'bad': 27374,
 'would': 27333,
 'veri': 26

### Building Features

Now let's build the features. Using the top 5,000 words, create a 2-dimensional matrix to record whether each of those words is contained in each document (tweet). Then you also have an output column to indicate whether the sentiment in each tweet is positive. For example, assuming your bag of words has 5 items (`['one', 'two', 'three', 'four', 'five']`) out of 4 documents (`['A', 'B', 'C', 'D']`), your feature set is essentially:

| Doc | one | two | three | four | five | is_positive |
|---|---|---|---|---|---|---|
| A | True | False | False | True | False | True |
| B | False | False | False | True | True | False |
| C | False | True | False | False | False | True |
| D | True | False | False | False | True | False|

However, because the `nltk.NaiveBayesClassifier.train` class we will use in the next step does not work with Pandas dataframe, the structure of your feature set should be converted to the Python list looking like below:

```python
[
	({
		'one': True,
		'two': False,
		'three': False,
		'four': True,
		'five': False
	}, True),
	({
		'one': False,
		'two': False,
		'three': False,
		'four': True,
		'five': True
	}, False),
	({
		'one': False,
		'two': True,
		'three': False,
		'four': False,
		'five': False
	}, True),
	({
		'one': True,
		'two': False,
		'three': False,
		'four': False,
		'five': True
	}, False)
]
```

To help you in this step, watch the [following video](https://www.youtube.com/watch?v=-vVskDsHcVc) to learn how to build the feature set with Python and NLTK. The source code in this video can be found [here](https://pythonprogramming.net/words-as-features-nltk-tutorial/).

[![Building Features](building-features.jpg)](https://www.youtube.com/watch?v=-vVskDsHcVc)

In [12]:
# your code here\

word_features = list(res.keys())

def find_features(document):
    words = document
    features = {}
    for w in word_features:
        features[w] = (w in words)
    return features

In [13]:
documents = tweets['text_processed']
categories = tweets.target.apply(lambda x: np.where(x==0, False, True))

In [15]:
featuresets = [(find_features(document), category) for (document, category) in zip(documents, categories)]

### Building and Traininng Naive Bayes Model

In this step you will split your feature set into a training and a test set. Then you will create a Bayes classifier instance using `nltk.NaiveBayesClassifier.train` ([example](https://www.nltk.org/book/ch06.html)) to train with the training dataset.

After training the model, call `classifier.show_most_informative_features()` to inspect the most important features. The output will look like:

```
Most Informative Features
	    snow = True            False : True   =     34.3 : 1.0
	  easter = True            False : True   =     26.2 : 1.0
	 headach = True            False : True   =     20.9 : 1.0
	    argh = True            False : True   =     17.6 : 1.0
	unfortun = True            False : True   =     16.9 : 1.0
	    jona = True             True : False  =     16.2 : 1.0
	     ach = True            False : True   =     14.9 : 1.0
	     sad = True            False : True   =     13.0 : 1.0
	  parent = True            False : True   =     12.9 : 1.0
	  spring = True            False : True   =     12.7 : 1.0
```

The [following video](https://www.youtube.com/watch?v=rISOsUaTrO4) will help you complete this step. The source code in this video can be found [here](https://pythonprogramming.net/naive-bayes-classifier-nltk-tutorial/).

[![Building and Training NB](nb-model-building.jpg)](https://www.youtube.com/watch?v=rISOsUaTrO4)

In [16]:
from nltk import NaiveBayesClassifier
from nltk.classify import accuracy

In [17]:
# your code here
training_set = featuresets[:15000]
test_set = featuresets[15000:]

5000

In [18]:
classifier = NaiveBayesClassifier.train(training_set)
# print(accuracy(classifier, test_set))

In [20]:
classifier.show_most_informative_features(10)

Most Informative Features
            followfriday = True             True : False  =     19.7 : 1.0
                unfortun = True            False : True   =     19.6 : 1.0
                 stomach = True            False : True   =     17.6 : 1.0
                  broken = True            False : True   =     17.0 : 1.0
                  throat = True            False : True   =     15.6 : 1.0
                    ouch = True            False : True   =     15.0 : 1.0
                 terribl = True            False : True   =     15.0 : 1.0
                     sad = True            False : True   =     13.7 : 1.0
                    poor = True            False : True   =     12.6 : 1.0
                  afford = True            False : True   =     11.0 : 1.0


### Testing Naive Bayes Model

Now we'll test our classifier with the test dataset. This is done by calling `nltk.classify.accuracy(classifier, test)`.

As mentioned in one of the tutorial videos, a Naive Bayes model is considered OK if your accuracy score is over 0.6. If your accuracy score is over 0.7, you've done a great job!

In [ ]:
# your code here
print(accuracy(classifier, test_set))

## Bonus Question 1: Improve Model Performance

If you are still not exhausted so far and want to dig deeper, try to improve your classifier performance. There are many aspects you can dig into, for example:

* Improve stemming and lemmatization. Inspect your bag of words and the most important features. Are there any words you should furuther remove from analysis? You can append these words to further remove to the stop words list.

* Remember we only used the top 5,000 features to build model? Try using different numbers of top features. The bottom line is to use as few features as you can without compromising your model performance. The fewer features you select into your model, the faster your model is trained. Then you can use a larger sample size to improve your model accuracy score.

In [ ]:
# your code here


## Bonus Question 2: Machine Learning Pipeline

In a new Jupyter Notebook, combine all your codes into a function (or a class). Your new function will execute the complete machine learning pipeline job by receiving the dataset location and output the classifier. This will allow you to use your function to predict the sentiment of any tweet in real time. 

In [ ]:
# your code here

## Bonus Question 3: Apache Spark

If you have completed the Apache Spark advanced topic lab, what you can do is to migrate your pipeline from local to a Databricks Notebook. Share your notebook with your instructor and classmates to show off your achievements!

In [ ]:
# your code here